<center> <img src = https://fotovmire.ru/wp-content/uploads/2019/05/16767/mashinist-i-stazhjor-v-kabine-lokomotiva.jpg alt="drawing" style="width:400px;">

## <center>Контроль за вниманием локомотивной бригады.

### <center>Постановка задачи
Создание системы контроля бдительности локомотивной бригады при управлении подвижным составом на основе нейросетевой аналитики, фиксация отвлечений от управления на мобильный телефон, личные электронные устройства, прочих отвлечений.

### <center>Проблематика
Работникам локомотивных бригад, в виду высокой степени ответственности их профессии, запрещено при ведении электропоезда пользоваться мобильными телефонами или иными гаджетами. Несмотря на это, периодически корневой причиной нарушений требований безопасности движения становится отвлечение локомотивной бригады от управления подвижным составом на мобильный телефон, личные электронные устройства и другие гаджеты.
В 2022 году на инфраструктуре ОАО «РЖД» было зафиксировано 1911 случаев нарушения безопасности движения. Из них число проездов запрещающего показания светофора составило 21 случай, число сходов подвижного состава 82 случая. Основной причиной данных видов нарушения безопасности движения является отвлечение от управления подвижным составом.
Предложенное решение позволит адресно проводить профилактическую работу с работниками локомотивных бригад в целях предотвращения отвлечений на мобильные телефоны.

### <center>Решение
Решение кейса представляет собой прототип системы, способной анализировать видеофайлы на предмет отвлечения локомотивной бригады от управления на посторонние предметы (основное использование телефона или любого другого гаджета) и формировать по результатам анализа отчёт.
Отчёт должен содержать: количество выявленных нарушений, типы нарушений, время нарушения на временной шкале видеофайла.
При этом система должна определять именно факт использования телефона, а не наличие его в кадре. Не является нарушение кратковременное использование телефона, к примеру, активация экрана до 3х секунд для ознакомления со временем.

Импортируем нужные для нас библиотеки.

In [14]:
import tqdm
import random
import pathlib
import itertools
import collections
import torch
import os
import cv2
import numpy as np
import remotezip as rz

import time
from collections import Counter
import logging
import tkinter as tk
import tensorflow as tf
import csv
import urllib.request
import imageio
from IPython import display
from urllib import request
from tensorflow_docs.vis import embed
import sys
import tarfile
from urllib.request import urlretrieve


Загрузим и распакуем данные из архива, из внешнего источника (в данном случае из архива PASCAL VOC 2012, который является общедоступным набором данных для задач компьютерного зрения)

In [17]:


def progress_bar(count, total, status=""):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = "=" * filled_len + "-" * (bar_len - filled_len)

    sys.stdout.write("[{}] {}%  {}\r".format(bar, percents, status))
    sys.stdout.flush()


def download_progress(block_num, block_size, total_size):
    progress_bar(block_num * block_size, total_size)


url = "http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar"
output_directory = "./data"
output_file = os.path.join(output_directory, "voc2012_raw.tar")
extract_directory = os.path.join(output_directory, "voc2012_raw")

# Создание папок, если они не существуют
os.makedirs(output_directory, exist_ok=True)
os.makedirs(extract_directory, exist_ok=True)

# Загрузка архива
urlretrieve(url, output_file, reporthook=download_progress)

# Извлечение содержимого архива
with tarfile.open(output_file, "r") as tar:
    tar.extractall(path=extract_directory)

# Вывод содержимого извлеченной папки
for item in os.listdir(os.path.join(extract_directory, "VOCdevkit/VOC2012")):
    print(item)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



ImageSets====================================================] 100.0%  
SegmentationClass
SegmentationObject
Annotations
JPEGImages


In [18]:

root = "sf_data_science/Vacation" 
data_root = os.path.join(root, "pew_research_data")

Открываем и считываем данные из CSV-файла, указанного в file_path, и возвращаем эти данные в виде списка словарей.

In [19]:

def load_file(file_path):
    data = []
    with open(file_path, encoding="utf-8") as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data.append(row)
    return data

Выполняем создание пути к файлу и его загрузку.

In [20]:
file_path = os.path.join(data_root, "sf_data_science/Vacation/submission.csv")
file_data = load_file(file_path)

Данный код используется для предобработки и загрузки набора данных изображений для задачи сегментации из представленного в torchvision набора данных PASCAL VOC.



In [21]:

from torchvision.datasets import VOCSegmentation
from torchvision.transforms import Compose, RandomHorizontalFlip

# Указываем дополнительные настройки, если это необходимо
seg_root = "/sf_data_science/Vacation"  # Замените '/some/path' на ваш путь к корневому каталогу данных
image_set = "train"
transformations = Compose([RandomHorizontalFlip(0.5)])

# Создание набора данных для сегментации
seg_dataset = VOCSegmentation(seg_root, year="2012", image_set=image_set, transforms=transformations, download=True)


100%|██████████| 1999639040/1999639040 [13:28<00:00, 2472487.87it/s] 


Extracting /Users/kakotichi/Documents/GitHub/sf_data_science/Vacation/VOCtrainval_11-May-2012.tar to /Users/kakotichi/Documents/GitHub/sf_data_science/Vacation


Готовим для обучения модели компьютерного зрения в библиотеке torchvision, которая используется для обучения моделей глубокого обучения в PyTorch. Более конкретно, этот код используется для загрузки и предварительной обработки набора данных VOCSegmentation, который является популярным датасетом для задач семантической сегментации изображений.

In [22]:

from torchvision.datasets import VOCSegmentation
from torchvision.transforms import Compose, RandomHorizontalFlip

# Указываем дополнительные настройки, если это необходимо
seg_root = "/sf_data_science/Vacation"  # Замените '/some/path' на ваш путь к корневому каталогу данных
image_set = "train"
transformations = Compose([RandomHorizontalFlip(0.5)])

# Создание набора данных для сегментации
seg_dataset = VOCSegmentation(seg_root, year="2012", image_set=image_set, transforms=transformations, download=True)

Using downloaded and verified file: /Users/kakotichi/Documents/GitHub/sf_data_science/Vacation/VOCtrainval_11-May-2012.tar
Extracting /Users/kakotichi/Documents/GitHub/sf_data_science/Vacation/VOCtrainval_11-May-2012.tar to /Users/kakotichi/Documents/GitHub/sf_data_science/Vacation


Создаем новый набор данных для обнаружения объектов, используя данные из набора данных Pascal VOC (Visual Object Classes) 2012. Это делается с помощью функции VOCDetection из библиотеки torchvision.datasets, которая загружает набор данных VOC.

Набор данных Pascal VOC включает изображения с объектами различных классов. Он часто используется для задач компьютерного зрения, таких как распознавание объектов и сегментация изображений.

In [23]:

from torchvision.datasets import VOCDetection
from torchvision.transforms import Compose, RandomHorizontalFlip

# Указываем дополнительные настройки, если это необходимо
det_root = "/sf_data_science/Vacation"  # Замените '/some/path' на ваш путь к корневому каталогу данных
image_set = "train"
transformations = Compose([RandomHorizontalFlip(0.5)])

# Создание набора данных для обнаружения объектов
det_dataset = VOCDetection(det_root, year="2012", image_set=image_set, transforms=transformations, download=True)


Using downloaded and verified file: /Users/kakotichi/Documents/GitHub/sf_data_science/Vacation/VOCtrainval_11-May-2012.tar
Extracting /Users/kakotichi/Documents/GitHub/sf_data_science/Vacation/VOCtrainval_11-May-2012.tar to /Users/kakotichi/Documents/GitHub/sf_data_science/Vacation


Создаем новый набор данных для глубокого обучения, объединяющий два разных набора данных из Pascal VOC: один набор данных для сегментации изображений (где каждому пикселю изображения присваивается семантическая метка класса), и другой для распознавания объектов (где каждому объекту на изображении присваивается ограничивающий прямоугольник и метка класса).

In [25]:
import torch
from torch.utils.data import Dataset
from torchvision.datasets import VOCSegmentation, VOCDetection

class CombinedVOC(Dataset):
    def __init__(self, seg_root, det_root, year, image_set, transforms=None, download=True):
        self.seg_dataset = VOCSegmentation(seg_root, year=year, image_set=image_set, transforms=transforms, download=download)
        self.det_dataset = VOCDetection(det_root, year=year, image_set=image_set, transforms=transforms, download=download)
        self.transforms = transforms

    def __len__(self):
        return min(len(self.seg_dataset), len(self.det_dataset))

    def __getitem__(self, idx):
        seg_data, seg_target = self.seg_dataset[idx]
        det_data, det_target = self.det_dataset[idx]

        if self.transforms is not None:
            seg_data, seg_target = self.transforms(seg_data, seg_target)
            det_data, det_target = self.transforms(det_data, det_target)

        return (seg_data, seg_target), (det_data, det_target)

seg_root = "path/to/seg_data"
det_root = "path/to/det_data"
year = "2012"
image_set = "trainval"
transformations = None
download = True

combined_dataset = CombinedVOC(seg_root, det_root, year, image_set, transforms=transformations, download=download)


100%|██████████| 1999639040/1999639040 [16:32<00:00, 2015403.74it/s] 


Extracting path/to/seg_data/VOCtrainval_11-May-2012.tar to path/to/seg_data


OSError: [Errno 28] No space left on device

Данный код выполняет следующие действия:

- Импортирует необходимые библиотеки (cv2 – для работы с изображениями, numpy – для работы с массивами данных, urllib.request – для выполнения запросов к web-ресурсам).

- Загружает конфигурацию и веса для модели YOLOv3 из указанных файлов. YOLO (You Only Look Once) – это алгоритм обработки изображений, который используется для обнаружения объектов.

- Загружает список классов, который может использоваться моделью YOLO для обнаружения и классификации объектов. Классы в данном случае - это типы объектов, которые YOLO может обнаружить (например, человек, автомобиль, собака и т.д.)

Тем не менее, сам код не выполняет детекцию объектов, он только загружает необходимые для этой операции файлы и модели.

In [26]:

# Загрузка модели и весов YOLOv3
yolo_config = "/Vacation/модель/yolov3.cfg"
yolo_weights = "/Vacation/модель/yolov3.weights"
net = cv2.dnn.readNet(yolo_weights, yolo_config)

# Загрузка списка классов
class_file = "/Vacation/data/coco.txt"
with open(class_file, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

 


По проходу через модель изображения программа будет обнаруживать и отслеживать только эти типы объектов. Это может быть полезно для приложений, которым необходимо фокусироваться только на определенных объектах и игнорировать все остальные.

In [27]:
# Получение выходных слоев для сети YOLO
layer_names = net.getLayerNames()
output_layers_indices = net.getUnconnectedOutLayers()
output_layers_indices = np.reshape(output_layers_indices, (output_layers_indices.size,))
out_layers = [layer_names[i - 1] for i in output_layers_indices]

# Устанавливаем классы, которые хотим обнаружить
classes_to_look_for = ["person", "cell phone", "laptop", "tv", "remote"] 


Далее прописываем код для обработки видео и выводим результат обработки видео с обнаружением телефона

In [8]:


def detect_objects(frame, net, out_layers, classes_to_look_for):
    detected_objects = []
    height, width, channels = frame.shape
    blob = cv2.dnn.blobFromImage(frame, scalefactor=1/255, size=(416, 416), mean=(0, 0, 0), swapRB=True, crop=False)
    net.setInput(blob)
    layer_outputs = net.forward(out_layers)

    class_ids = []
    confidences = []
    boxes = []

    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.4 and classes[class_id] in classes_to_look_for:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                confidences.append(float(confidence))
                boxes.append([x, y, w, h])
                class_ids.append(class_id)

    indices = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold=0.4, nms_threshold=0.5)
    
    if isinstance(indices, tuple) and not indices:
         indices = []
    elif isinstance(indices, np.ndarray):
        indices = indices.astype(int)
        indices = indices.reshape(-1, 1).tolist()
    else:
        raise ValueError(f"Unexpected value in 'indices': {indices}")

    for i in indices:
        i = i[0]
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        detected_objects.append(label)
        confidence = round(confidences[i], 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, f'{label} {confidence}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)  
    return frame, detected_objects
yolo_config = "/Vacation/модель/yolov3.cfg"
yolo_weights = "/Vacation/модель/yolov3.weights"
net = cv2.dnn.readNet(yolo_weights, yolo_config)

class_file = "/Vacation/data/coco.txt"
with open(class_file, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers_indices = net.getUnconnectedOutLayers()
output_layers_indices = np.reshape(output_layers_indices, (output_layers_indices.size,))
out_layers = [layer_names[i - 1] for i in output_layers_indices]

classes_to_look_for = ["person", "cell phone", "laptop", "tv", "remote"]
# Настройка логирования
logging.basicConfig(filename='results.log', level=logging.INFO)

def load_video():
    return "train_dataset_Бригады/Анализ бригад (телефон)/Есть телефон/22_11_46.mp4"

video_path = load_video()  # video_path теперь определён до его первого использования
cap = cv2.VideoCapture(video_path)

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')


class_file = "/Vacation/data/coco.txt"
with open(class_file, 'r') as f:
    classes = [line.strip() for line in f.readlines()]


output_folder = "/Vacation/result"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
output_video_path = os.path.join(output_folder, 'результат.mp4')

out = cv2.VideoWriter(output_video_path, fourcc, 20.0, (frame_width,frame_height))

phone_in_hands = False
phone_detection_timer = None
phone_hold_time = 3.0

events = []  # список для сохранения всех событий

while cap.isOpened():
    ret, frame = cap.read()

    if ret:
        processed_frame, detected_objects = detect_objects(frame, net, out_layers, classes_to_look_for)
        detected_objects_count = Counter(detected_objects)

        detected_phone_count = detected_objects_count.get('cell phone', 0)
        detected_person_count = detected_objects_count.get('person', 0)

        if detected_phone_count > 0:
            if not phone_in_hands:
                phone_detection_timer = time.time()
                logging.info('Обнаружено событие: начало использования телефона')
            phone_in_hands = True
        else:
            if phone_in_hands:
                 logging.info('Обнаружено событие: окончание использования телефона')
            phone_in_hands = False


        if detected_person_count >= 2:
            if phone_in_hands and (time.time() - phone_detection_timer) >= phone_hold_time:
                event_msg = f"Телефон(ы) был(и) в руках двух человек более {phone_hold_time} секунд."
                events.append(event_msg)
        elif detected_person_count == 1:
            if phone_in_hands and (time.time() - phone_detection_timer) >= phone_hold_time:
                event_msg = "Телефон был в руках одного человека более 3-х секунд."
                events.append(event_msg)
       
        cv2.imshow('Detected Objects', processed_frame)
        out.write(processed_frame)

        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
out.release()
cv2.destroyAllWindows()

print("\nОбработка видео завершена. Обнаруженные события:")
for idx, event in enumerate(events, start=1):
    logging.info(f"Обнаружено событие {idx}: {event}")
    print(f"{idx}. {event}")



Обработка видео завершена. Обнаруженные события:


### <center>Вывод

Из-за неправильно выбранной модели ИИ, мы можем наблюдать не совсем корректную обработку видео.

<image src="result/Снимок экрана 2023-09-24 в 01.31.51.png" alt="Текст с описанием картинки">

<image src="result/Снимок экрана 2023-10-09 в 12.57.08.png" alt="Текст с описанием картинки">


<video controls>
  <source src="/Users/kakotichi/Documents/GitHub/sf_data_science/Vacation/result/Запись экрана 2023-10-09 в 12.56.27.mov">
  Ваш браузер не поддерживает воспроизведение видео.
</video>


<video controls>
  <source src="/Users/kakotichi/Documents/GitHub/sf_data_science/Vacation/result/Запись экрана 2023-10-09 в 13.05.39.mov">
  Ваш браузер не поддерживает воспроизведение видео.
</video>

Наша выбранная модель Yolo3 работает вполне не плохо, но для корректной обработки видео стоило выбирать модель Yolo8, который более корректно работает, чем наша модель, т.к она обучена на большем количестве данных! Но мы видем, что в целом наша модель вполне не плохо справляется с обнаружением телефона на видео. И результат обработки видео мы можеи видеть в протоколе под названием results.log

### Так же мы можем посчитать среднее значение для каждого класса данных

In [7]:
average_counts = {class_name:[] for class_name in classes_to_look_for}

while cap.isOpened():
    ret, frame = cap.read()

    if ret:

        processed_frame, detected_objects = detect_objects(frame, net, out_layers, classes_to_look_for)

        # Подсчёт обнаруженных объектов
        detected_objects_count = Counter(detected_objects)

        # Сохранение количества для среднего значения
        for class_name in classes_to_look_for:
            average_counts[class_name].append(detected_objects_count.get(class_name, 0))

        detected_phone_count = detected_objects_count.get('cell phone', 0)
        detected_person_count = detected_objects_count.get('person', 0)

        current_time_secs = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0 

        if detected_phone_count > 0:
            if not phone_in_hands:
                phone_detection_timer = current_time_secs
                logging.info('Обнаружено событие: начало использования телефона')
            phone_in_hands = True
        else:
            if phone_in_hands:
                logging.info('Обнаружено событие: окончание использования телефона')
            phone_in_hands = False

        if detected_person_count >= 2:
            if phone_in_hands and (current_time_secs - phone_detection_timer) >= phone_hold_time:
                event_msg = f"Телефон(ы) был(и) в руках двух человек более {phone_hold_time} секунд."
                events.append(event_msg)
        elif detected_person_count == 1:
            if phone_in_hands and (current_time_secs - phone_detection_timer) >= phone_hold_time:
                event_msg = "Телефон был в руках одного человека более 3-х секунд."
                events.append(event_msg)

        # Here we draw the frame and write to the video file
        cv2.imshow('Detected Objects', processed_frame)
        out.write(processed_frame)

        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break

# Calculating average counts
average_counts = {class_name: sum(counts)/len(counts) for class_name, counts in average_counts.items()}
print(f"Average Counts: {average_counts}")

Average Counts: {'person': 0.0, 'cell phone': 1.394621372965322, 'laptop': 0.0, 'tv': 0.0, 'remote': 0.0}


Здесь мы можем наблюдать некоторую погрешность, ведь один или несколько человек всегда присуствует в кадре. А вот  определение телефона в среднем в данныых находит 1.4 мобильного телефона 